# K-均值聚类算法

**优点：**容易实现。  
**缺点：**可能收敛到局部最小值，在大规模数据集上收敛较慢。   
**适用数据类型：**数值型数据。

伪代码：  
创建$k$个点作为起始质心（经常是随机选择）  
当任意一个点的簇分配结构发生改变时  
&emsp;&emsp;对数据集中的每个数据点  
&emsp;&emsp;&emsp;&emsp;对每个质心  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;计算质心与数据点之间的距离  
&emsp;&emsp;&emsp;&emsp;将数据点分配到距其最近的簇  
&emsp;&emsp;对每一个簇，计算簇中所有点的均值并将均值作为质心

**一般流程**  
1. 收集数据：使用任意方法。
2. 准备数据：需要数值型数据来计算距离，也可以将标称型数据映射为二值型数据再用于距离计算。
3. 分析数据：使用任意方法。
4. 训练算法：不适用于无监督学习，即无监督学习没有训练过程。
5. 测试算法：应用聚类算法、观察结果。可以使用量化的误差指标如误差平方和（后面会介绍）来评价算法的结果。
6. 使用算法：可以用于所希望的任何应用。通常情况下，簇质心可以代表整个簇的数据来做出决策。

In [1]:
import numpy as np

## K-均值聚类支持函数

In [2]:
# 将文本导入到一个列表中。
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine))
        dataMat.append(fltLine)
    return dataMat

两个向量的欧式距离公式：$d_{12}=\sqrt{\sum_{k=1}^n(x_{1k}-x_{2k})^2}$

In [3]:
# 计算两个向量的欧式距离
def distEclud(vecA, vecB):
    return np.sqrt(np.sum(np.power(vecA - vecB, 2)))

In [4]:
# 为给定数据集构建一个包含k个随机质心的集合。
# 随机质心必须要在整个数据集的边界之内。
def randCent(dataSet, k):
    # 获得dataSet的维度
    n = np.shape(dataSet)[1]
    # 创建k行n列的向量
    centroids = np.mat(np.zeros((k, n)))
    for j in range(n):
        # 获取该列的最小值
        minJ = min(dataSet[:, j])
        # 获取该列的范围
        rangeJ = float(max(dataSet[:, j]) - minJ)
        # 获取随机的列数据，并确保在边界值内
        centroids[:, j] = minJ + rangeJ * np.random.rand(k, 1)
    return centroids

### 测试三个函数的实际效果

In [5]:
datMat = np.mat(loadDataSet('testSet.txt'))

In [6]:
min(datMat[:, 0])

matrix([[-5.379713]])

In [7]:
min(datMat[:, 1])

matrix([[-4.232586]])

In [8]:
max(datMat[:, 1])

matrix([[5.1904]])

In [9]:
max(datMat[:, 0])

matrix([[4.838138]])

In [10]:
randCent(datMat, 2)

matrix([[-3.20342207,  1.66863242],
        [-4.72757408,  5.07705288]])

In [11]:
distEclud(datMat[0], datMat[1])

5.184632816681332

准备实现完整的K-均值算法。该算法会常见$k$个质心，然后将每个点分配到最近的质心，再重新计算质心。这个过程重复数次，直到数据点的簇分配结果不再改变为止。

## K-均值聚类算法

In [12]:
# dataSet: 数据集，k: 簇的数目， distMeas: 用来计算距离的函数，createCent: 用来创建初始质心的函数
def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    # 确定数据集中数据点的总数
    m = np.shape(dataSet)[0]
    # 创建一个矩阵来存储每个点的簇分配结果
    # 包括两列： 第一列记录簇索引值，第二列存储误差（当前点到簇质心的距离）
    clusterAssment = np.mat(np.zeros((m, 2)))
    # 初始化创建k个质心
    centroids = createCent(dataSet, k)
    # 设置聚类是否改变的标志变量
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = np.inf; minIndex = -1
            # 寻找最近的质心
            for j in range(k):
                distJI = distMeas(centroids[j, :], dataSet[i, :])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            # 判断是否最近质心变了
            if clusterAssment[i, 0] != minIndex:
                clusterChanged = True
            # 对误差取平方的目的是更加重视那些远离中心的点
            clusterAssment[i, :] = minIndex, minDist**2
        print(centroids)
        # 更新质心的位置
        for cent in range(k):
            ptsInClust = dataSet[np.nonzero(clusterAssment[:, 0].A == cent)[0]]
            centroids[cent, :] = np.mean(ptsInClust, axis=0)
    # centroids：质心矩阵，clusterAssment：每个点的簇的分配结果
    return centroids, clusterAssment

In [13]:
datMat = np.mat(loadDataSet('testSet.txt'))

In [14]:
myCentroids, clustAssing = kMeans(datMat, 4)

[[-0.12632757 -3.54832673]
 [ 1.47861736  1.27753822]
 [-0.03262045  1.17126921]
 [ 0.7809183   4.45044345]]
[[-0.72175297 -3.03774939]
 [ 3.36858169  1.49441523]
 [-2.46571578  2.45059206]
 [ 1.40656708  3.85884762]]
[[-0.83188333 -2.97988206]
 [ 3.67314583  0.937859  ]
 [-2.46154315  2.78737555]
 [ 2.23432133  3.71816925]]
[[-1.05611215 -3.00107638]
 [ 3.6509195  -0.5281174 ]
 [-2.46154315  2.78737555]
 [ 2.5212765   3.49464725]]
[[-2.79969165 -3.01951378]
 [ 3.1666855  -2.38897356]
 [-2.46154315  2.78737555]
 [ 2.54391447  3.21299611]]
[[-3.38237045 -2.9473363 ]
 [ 2.80293085 -2.7315146 ]
 [-2.46154315  2.78737555]
 [ 2.6265299   3.10868015]]


## 二分K-均值算法

**算法的伪代码：**  
将所有点看成一个簇  
当簇数目小于$k$时  
&emsp;&emsp;对于每一个簇  
&emsp;&emsp;&emsp;&emsp;计算总误差  
&emsp;&emsp;&emsp;&emsp;在给定的簇上面进行K-均值聚类($k$=2)  
&emsp;&emsp;&emsp;&emsp;计算将该簇一分为二之后的总误差  
&emsp;&emsp;选择使得误差最小的那个簇进行划分操作  

In [15]:
# dataSet: 数据集，k: 簇的数目，distMeas：用来计算距离的函数
def biKmeans(dataSet, k, distMeas=distEclud):
    # 确定数据集中数据点的总数
    m = np.shape(dataSet)[0]
    # 创建一个矩阵来存储数据集中每个点的簇分配结果和平方误差
    clusterAssment = np.mat(np.zeros((m,2)))
    # 计算整个数据集的质心
    centroid0 = np.mean(dataSet, axis=0).tolist()[0]
    # 使用一个列表保存所有质心
    centList =[centroid0]
    # 计算每个点到质心的误差值
    for j in range(m):#calc initial Error
        clusterAssment[j,1] = distMeas(np.mat(centroid0), dataSet[j,:])**2
    # 对簇进行划分，直到得到想要的簇数目为止
    while (len(centList) < k):
        # 将最小的SEE设置为无穷大
        lowestSSE = np.inf
        for i in range(len(centList)):
            # 将该簇中的所有点看成一个小的数据集
            ptsInCurrCluster = dataSet[np.nonzero(clusterAssment[:,0].A==i)[0],:]#get the data points currently in cluster i
            # 将ptsInCurrCluster输入到函数kMeans()中进行处理k=2，得到两个质心，并给出每个簇的误差值
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)
            # 计算当前的误差之和
            sseSplit = np.sum(splitClustAss[:,1])#compare the SSE to the currrent minimum
            # 计算剩余数据集的误差之后
            sseNotSplit = np.sum(clusterAssment[np.nonzero(clusterAssment[:,0].A!=i)[0],1])
            print("sseSplit, and notSplit: ",sseSplit,sseNotSplit)
            # 如果划分的SSE值最小，则保留本次划分
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        # 得到两个编号为0和1的结果簇，需要将这些簇编号修改为划分簇及新加簇的编号
        bestClustAss[np.nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList) #change 1 to 3,4, or whatever
        bestClustAss[np.nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        print('the bestCentToSplit is: ',bestCentToSplit)
        print('the len of bestClustAss is: ', len(bestClustAss))
        # 新的簇分配结果被更新，新的质心会被添加到centList中
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]#replace a centroid with two best centroids 
        centList.append(bestNewCents[1,:].tolist()[0])
        clusterAssment[np.nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:]= bestClustAss#reassign new clusters, and SSE
    return np.mat(centList), clusterAssment

In [16]:
datMat3 = np.mat(loadDataSet('testSet2.txt'))

In [19]:
centList, myNewAssments = biKmeans(datMat3, 3)

[[-2.35094679  2.2704867 ]
 [-0.18528081 -1.77306899]]
[[-0.48729809  3.42433234]
 [ 0.30368272 -1.853273  ]]
[[-0.06953469  3.29844341]
 [-0.32150057 -2.62473743]]
[[-0.00675605  3.22710297]
 [-0.45965615 -2.7782156 ]]
sseSplit, and notSplit:  453.0334895807502 0.0
the bestCentToSplit is:  0
the len of bestClustAss is:  60
[[-1.06541045  4.32355066]
 [ 0.32170111  2.01795651]]
[[-2.94737575  3.3263781 ]
 [ 2.93386365  3.12782785]]
sseSplit, and notSplit:  77.59224931775066 29.15724944412535
[[-1.0764188  -1.40525693]
 [-0.60261464 -1.9079172 ]]
[[-1.3776246  -1.6522424 ]
 [-0.15366667 -3.15354   ]]
[[-1.41084317 -1.873139  ]
 [-0.05200457 -3.16610557]]
[[-1.31198114e+00 -1.96162114e+00]
 [-7.11923077e-04 -3.21792031e+00]]
[[-1.26873575 -2.07139688]
 [ 0.07973025 -3.24942808]]
[[-1.26405367 -2.209896  ]
 [ 0.19848727 -3.24320436]]
[[-1.1836084 -2.2507069]
 [ 0.2642961 -3.3057243]]
[[-1.12616164 -2.30193564]
 [ 0.35496167 -3.36033556]]
sseSplit, and notSplit:  12.753263136887313 423.876

查看质心结果：

In [20]:
centList

matrix([[-2.94737575,  3.3263781 ],
        [-0.45965615, -2.7782156 ],
        [ 2.93386365,  3.12782785]])